# Walk-Forward Diagnostic Analysis

This notebook investigates why our models fail in walk-forward validation:
- **Single-split Sharpe**: +0.301
- **Walk-forward Sharpe**: -0.969

Goals:
1. Identify when/where models fail (time periods, regimes)
2. Analyze prediction quality (rank correlation, hit rate trends)
3. Understand feature effectiveness
4. Guide feature engineering efforts

In [ ]:
import sys
sys.path.insert(0, '..')

import json
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from scipy import stats

from qcml_rotation.analysis.diagnostics import (
    analyze_time_periods,
    analyze_prediction_quality,
    compute_rank_correlation,
    analyze_regime_performance,
    compute_rolling_metrics,
    compute_train_test_gap,
    create_diagnostic_report
)
from qcml_rotation.backtest.metrics import (
    bootstrap_sharpe_ci,
    sharpe_p_value,
    permutation_test,
    compute_significance
)

# Set style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

print("Diagnostic analysis ready")

## 1. Load Data and Results

In [ ]:
# Load processed data
with open('../data/processed/processed_data.pkl', 'rb') as f:
    data = pickle.load(f)

splits = data['splits']
feature_cols = data['feature_cols']
prices = data['prices']
rebalance_dates = data['rebalance_dates']

print(f"Feature columns: {feature_cols}")
print(f"Total weeks: {len(rebalance_dates)}")
print(f"Date range: {rebalance_dates[0]} to {rebalance_dates[-1]}")

In [ ]:
# Load walk-forward results if available
wf_path = Path('../outputs/walk_forward/walk_forward_results.json')

if wf_path.exists():
    with open(wf_path, 'r') as f:
        wf_results = json.load(f)
    print("Loaded walk-forward results")
    print(f"Models: {list(wf_results['models'].keys())}")
else:
    print("Walk-forward results not found. Run the script first:")
    print("  python scripts/run_walk_forward.py")
    wf_results = None

In [ ]:
# Load single-split backtest results for comparison
with open('../outputs/backtest/backtest_results.json', 'r') as f:
    bt_results = json.load(f)

print("Single-split test results:")
for name, result in bt_results.items():
    sharpe = result['metrics']['sharpe_ratio']
    ret = result['metrics']['total_return'] * 100
    print(f"  {name}: Sharpe={sharpe:.3f}, Return={ret:.1f}%")

## 2. Train/Test Performance Gap Analysis

Key question: Is the model overfitting to training data?

In [ ]:
# Analyze feature distributions in train vs test
train_data = splits.train
test_data = splits.test

print("Feature distribution comparison (Train vs Test):")
print("=" * 60)

distribution_stats = []
for col in feature_cols:
    train_mean = train_data[col].mean()
    train_std = train_data[col].std()
    test_mean = test_data[col].mean()
    test_std = test_data[col].std()
    
    # Compute distribution shift
    mean_shift = (test_mean - train_mean) / train_std if train_std > 0 else 0
    
    distribution_stats.append({
        'Feature': col,
        'Train Mean': train_mean,
        'Train Std': train_std,
        'Test Mean': test_mean,
        'Test Std': test_std,
        'Mean Shift (std)': mean_shift
    })

dist_df = pd.DataFrame(distribution_stats)
dist_df = dist_df.sort_values('Mean Shift (std)', key=abs, ascending=False)
print(dist_df.to_string(index=False))

In [ ]:
# Visualize distribution shifts
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for i, col in enumerate(feature_cols[:6]):
    ax = axes[i]
    ax.hist(train_data[col], bins=50, alpha=0.5, label='Train', density=True)
    ax.hist(test_data[col], bins=50, alpha=0.5, label='Test', density=True)
    ax.set_title(col)
    ax.legend()

plt.suptitle('Feature Distribution: Train vs Test', fontsize=14)
plt.tight_layout()
plt.savefig('../outputs/backtest/feature_distributions.png', dpi=150)
plt.show()

## 3. Time Period Analysis

When does the model fail? Yearly, quarterly, specific periods?

In [ ]:
# Analyze single-split results by time period
best_model = 'qcml_real_only'
returns = np.array(bt_results[best_model]['returns'])
dates = pd.to_datetime(bt_results[best_model]['dates'])

yearly, quarterly, worst = analyze_time_periods(returns, list(dates), n_worst=10)

print("Yearly Returns:")
print(yearly.to_string())
print("\nQuarterly Returns:")
print(quarterly.tail(8).to_string())

In [ ]:
# Visualize yearly performance
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Yearly returns bar chart
colors = ['green' if x > 0 else 'red' for x in yearly.values]
axes[0].bar(yearly.index.astype(str), yearly.values * 100, color=colors)
axes[0].axhline(y=0, color='black', linestyle='-', linewidth=0.5)
axes[0].set_xlabel('Year')
axes[0].set_ylabel('Return (%)')
axes[0].set_title(f'Yearly Returns: {best_model}')

# Quarterly returns
q_colors = ['green' if x > 0 else 'red' for x in quarterly.values]
axes[1].bar(range(len(quarterly)), quarterly.values * 100, color=q_colors)
axes[1].axhline(y=0, color='black', linestyle='-', linewidth=0.5)
axes[1].set_xlabel('Quarter')
axes[1].set_ylabel('Return (%)')
axes[1].set_title(f'Quarterly Returns: {best_model}')

plt.tight_layout()
plt.savefig('../outputs/backtest/time_period_returns.png', dpi=150)
plt.show()

In [ ]:
# Worst periods analysis
print("\nWorst 10 Periods (4-week rolling):")
print(worst.to_string())

# What was happening in the market during these periods?
print("\n" + "="*60)
print("Market Context During Worst Periods")
print("="*60)

spy_returns = prices['SPY'].pct_change()
for date in worst.index[:5]:
    if date in spy_returns.index:
        spy_ret = spy_returns.loc[date]
        model_ret = worst.loc[date, 'weekly_return']
        print(f"{date.date()}: Model={model_ret*100:.2f}%, SPY={spy_ret*100:.2f}%")

## 4. Prediction Quality Analysis

How good are the model's predictions at ranking ETFs?

In [ ]:
# Load predictions from backtest
# Note: We need the raw predictions - these might not be saved
# Let's reconstruct from model outputs

# Load model and make predictions on test set
from qcml_rotation.data.dataset import ETFDataset
import torch

test_dataset = ETFDataset(splits.test, feature_cols)
X_test = test_dataset.features.numpy()
y_test = test_dataset.labels.numpy()

print(f"Test set shape: X={X_test.shape}, y={y_test.shape}")
print(f"Number of tickers: {len(test_dataset.tickers)}")
print(f"Tickers: {test_dataset.tickers}")

In [ ]:
# Load trained model
with open('../outputs/qcml/qcml_models.pkl', 'rb') as f:
    qcml_models = pickle.load(f)

model = qcml_models['qcml_real_only']
model.eval()

# Make predictions
with torch.no_grad():
    X_tensor = torch.FloatTensor(X_test)
    predictions = model(X_tensor).numpy()

print(f"Predictions shape: {predictions.shape}")
print(f"Prediction range: [{predictions.min():.4f}, {predictions.max():.4f}]")
print(f"Prediction std: {predictions.std():.4f}")

In [ ]:
# Reshape predictions and actuals to (n_weeks, n_etfs)
n_tickers = len(test_dataset.tickers)
n_weeks = len(predictions) // n_tickers

# Get unique dates and reshape
test_dates_unique = splits.test.index.get_level_values('date').unique().sort_values()
n_weeks_actual = len(test_dates_unique)

print(f"Samples: {len(predictions)}, Tickers: {n_tickers}, Weeks: {n_weeks_actual}")

# Reshape to (weeks, tickers)
predictions_2d = predictions.reshape(n_weeks_actual, n_tickers)
actuals_2d = y_test.reshape(n_weeks_actual, n_tickers)

print(f"Reshaped: predictions={predictions_2d.shape}, actuals={actuals_2d.shape}")

In [ ]:
# Analyze prediction quality
pred_quality = analyze_prediction_quality(predictions_2d, actuals_2d, top_k=3)

print("Prediction Quality Metrics:")
print("="*60)
for key, value in pred_quality.items():
    if isinstance(value, float):
        print(f"{key}: {value:.4f}")
    else:
        print(f"{key}: {value}")

In [ ]:
# Compute weekly rank correlations
weekly_correlations = []
for week in range(n_weeks_actual):
    pred_week = predictions_2d[week]
    actual_week = actuals_2d[week]
    
    if np.std(pred_week) > 0 and np.std(actual_week) > 0:
        corr, _ = stats.spearmanr(pred_week, actual_week)
        if np.isfinite(corr):
            weekly_correlations.append(corr)
        else:
            weekly_correlations.append(0)
    else:
        weekly_correlations.append(0)

weekly_correlations = np.array(weekly_correlations)

print(f"Weekly Rank Correlations:")
print(f"  Mean: {weekly_correlations.mean():.4f}")
print(f"  Std: {weekly_correlations.std():.4f}")
print(f"  Min: {weekly_correlations.min():.4f}")
print(f"  Max: {weekly_correlations.max():.4f}")
print(f"  % Positive: {(weekly_correlations > 0).mean()*100:.1f}%")

In [ ]:
# Visualize rank correlation over time
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Weekly correlation time series
ax = axes[0, 0]
ax.plot(test_dates_unique, weekly_correlations, alpha=0.7)
ax.axhline(y=0, color='red', linestyle='--', linewidth=1)
ax.axhline(y=weekly_correlations.mean(), color='green', linestyle='--', 
           label=f'Mean: {weekly_correlations.mean():.3f}')
ax.fill_between(test_dates_unique, 0, weekly_correlations, 
                where=weekly_correlations > 0, alpha=0.3, color='green')
ax.fill_between(test_dates_unique, 0, weekly_correlations, 
                where=weekly_correlations <= 0, alpha=0.3, color='red')
ax.set_xlabel('Date')
ax.set_ylabel('Spearman Rank Correlation')
ax.set_title('Weekly Prediction Rank Correlation')
ax.legend()

# Histogram of correlations
ax = axes[0, 1]
ax.hist(weekly_correlations, bins=30, edgecolor='black', alpha=0.7)
ax.axvline(x=0, color='red', linestyle='--', linewidth=2)
ax.axvline(x=weekly_correlations.mean(), color='green', linestyle='--', linewidth=2)
ax.set_xlabel('Rank Correlation')
ax.set_ylabel('Frequency')
ax.set_title('Distribution of Weekly Rank Correlations')

# Rolling average correlation
ax = axes[1, 0]
rolling_corr = pd.Series(weekly_correlations, index=test_dates_unique).rolling(12).mean()
ax.plot(test_dates_unique, rolling_corr, color='blue', linewidth=2)
ax.axhline(y=0, color='red', linestyle='--')
ax.set_xlabel('Date')
ax.set_ylabel('12-Week Rolling Avg Correlation')
ax.set_title('Rolling Average Rank Correlation')

# Prediction vs Actual scatter (flattened)
ax = axes[1, 1]
ax.scatter(predictions_2d.flatten(), actuals_2d.flatten(), alpha=0.1, s=5)
ax.axhline(y=0, color='gray', linestyle='--', alpha=0.5)
ax.axvline(x=0, color='gray', linestyle='--', alpha=0.5)
# Add regression line
z = np.polyfit(predictions_2d.flatten(), actuals_2d.flatten(), 1)
p = np.poly1d(z)
x_line = np.linspace(predictions_2d.min(), predictions_2d.max(), 100)
ax.plot(x_line, p(x_line), 'r-', linewidth=2, label=f'Fit: y={z[0]:.3f}x+{z[1]:.3f}')
ax.set_xlabel('Predicted Return')
ax.set_ylabel('Actual Return')
ax.set_title('Prediction vs Actual Returns')
ax.legend()

plt.tight_layout()
plt.savefig('../outputs/backtest/prediction_quality.png', dpi=150)
plt.show()

## 5. Regime Analysis

Does the model perform differently in different market regimes?

In [ ]:
# Compute SPY returns for regime classification
spy_returns = prices['SPY'].pct_change().dropna()

# Align with test dates
test_dates_list = list(test_dates_unique)
spy_test_returns = spy_returns.loc[spy_returns.index.isin(test_dates_list)].values

# Analyze regime performance
regime_stats = analyze_regime_performance(
    returns, 
    list(dates),
    spy_returns=spy_test_returns[:len(returns)] if len(spy_test_returns) >= len(returns) else None
)

print("Performance by Regime:")
print(regime_stats.to_string(index=False))

In [ ]:
# Visualize regime performance
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# By volatility regime
vol_regime = regime_stats[regime_stats['regime_type'] == 'vol_regime']
if len(vol_regime) > 0:
    ax = axes[0]
    colors = ['green' if x > 0 else 'red' for x in vol_regime['mean_return']]
    bars = ax.bar(vol_regime['regime_value'], vol_regime['mean_return'] * 100, color=colors)
    ax.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
    ax.set_xlabel('Volatility Regime')
    ax.set_ylabel('Mean Return (%)')
    ax.set_title('Performance by Volatility Regime')
    
    # Add week counts
    for bar, weeks in zip(bars, vol_regime['n_weeks']):
        ax.text(bar.get_x() + bar.get_width()/2, bar.get_height(),
                f'n={weeks}', ha='center', va='bottom')

# By market regime (if available)
market_regime = regime_stats[regime_stats['regime_type'] == 'market_regime']
if len(market_regime) > 0:
    ax = axes[1]
    colors = ['green' if x > 0 else 'red' for x in market_regime['mean_return']]
    bars = ax.bar(market_regime['regime_value'], market_regime['mean_return'] * 100, color=colors)
    ax.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
    ax.set_xlabel('Market Regime')
    ax.set_ylabel('Mean Return (%)')
    ax.set_title('Performance by Market Regime')
    
    for bar, weeks in zip(bars, market_regime['n_weeks']):
        ax.text(bar.get_x() + bar.get_width()/2, bar.get_height(),
                f'n={weeks}', ha='center', va='bottom')
else:
    axes[1].text(0.5, 0.5, 'Market regime data not available', 
                 ha='center', va='center', transform=axes[1].transAxes)

plt.tight_layout()
plt.savefig('../outputs/backtest/regime_performance.png', dpi=150)
plt.show()

## 6. Rolling Performance Metrics

In [ ]:
# Compute rolling metrics
rolling = compute_rolling_metrics(returns, list(dates), window=26)

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Rolling Sharpe
ax = axes[0, 0]
ax.plot(rolling.index, rolling['rolling_sharpe'], color='blue')
ax.axhline(y=0, color='red', linestyle='--')
ax.axhline(y=1, color='green', linestyle='--', alpha=0.5)
ax.axhline(y=-1, color='red', linestyle='--', alpha=0.5)
ax.fill_between(rolling.index, 0, rolling['rolling_sharpe'], 
                where=rolling['rolling_sharpe'] > 0, alpha=0.3, color='green')
ax.fill_between(rolling.index, 0, rolling['rolling_sharpe'], 
                where=rolling['rolling_sharpe'] <= 0, alpha=0.3, color='red')
ax.set_xlabel('Date')
ax.set_ylabel('Sharpe Ratio')
ax.set_title('26-Week Rolling Sharpe Ratio')

# Rolling Return
ax = axes[0, 1]
ax.plot(rolling.index, rolling['rolling_return'] * 100, color='blue')
ax.axhline(y=0, color='red', linestyle='--')
ax.set_xlabel('Date')
ax.set_ylabel('Annualized Return (%)')
ax.set_title('26-Week Rolling Annualized Return')

# Cumulative Return
ax = axes[1, 0]
ax.plot(rolling.index, (rolling['cumulative'] - 1) * 100, color='blue')
ax.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
ax.set_xlabel('Date')
ax.set_ylabel('Cumulative Return (%)')
ax.set_title('Cumulative Return')

# Drawdown
ax = axes[1, 1]
ax.fill_between(rolling.index, rolling['drawdown'] * 100, 0, color='red', alpha=0.5)
ax.set_xlabel('Date')
ax.set_ylabel('Drawdown (%)')
ax.set_title('Drawdown')

plt.tight_layout()
plt.savefig('../outputs/backtest/rolling_metrics.png', dpi=150)
plt.show()

## 7. Feature Importance Analysis

Which features are most predictive?

In [ ]:
# Simple correlation-based feature importance
feature_correlations = []

for i, col in enumerate(feature_cols):
    feature_vals = X_test[:, i]
    corr, p_val = stats.pearsonr(feature_vals, y_test)
    spearman_corr, sp_p_val = stats.spearmanr(feature_vals, y_test)
    
    feature_correlations.append({
        'Feature': col,
        'Pearson Corr': corr,
        'Pearson P-val': p_val,
        'Spearman Corr': spearman_corr,
        'Spearman P-val': sp_p_val,
        'Significant': p_val < 0.05
    })

feat_corr_df = pd.DataFrame(feature_correlations)
feat_corr_df = feat_corr_df.sort_values('Pearson Corr', key=abs, ascending=False)
print("Feature Correlations with Forward Returns:")
print(feat_corr_df.to_string(index=False))

In [ ]:
# Visualize feature correlations
fig, ax = plt.subplots(figsize=(10, 6))

colors = ['green' if x else 'gray' for x in feat_corr_df['Significant']]
bars = ax.barh(feat_corr_df['Feature'], feat_corr_df['Pearson Corr'], color=colors)
ax.axvline(x=0, color='black', linestyle='-', linewidth=0.5)
ax.set_xlabel('Pearson Correlation')
ax.set_title('Feature Correlation with Forward Returns\n(Green = Significant at 5%)')

plt.tight_layout()
plt.savefig('../outputs/backtest/feature_importance.png', dpi=150)
plt.show()

## 8. Statistical Significance Analysis

In [ ]:
# Compute significance metrics
sig = compute_significance(returns, n_bootstrap=1000, random_state=42)

print("Statistical Significance Analysis:")
print("="*60)
print(f"Sharpe Ratio: {sig.sharpe_ratio:.4f}")
print(f"95% CI: [{sig.sharpe_ci_lower:.4f}, {sig.sharpe_ci_upper:.4f}]")
print(f"P-value (Sharpe > 0): {sig.sharpe_p_value:.4f}")
print(f"Statistically Significant: {'YES' if sig.is_significant else 'NO'}")

In [ ]:
# Permutation test for prediction skill
observed_corr, perm_p_val, null_dist = permutation_test(
    predictions_2d, actuals_2d, n_permutations=1000, random_state=42
)

print(f"\nPermutation Test for Prediction Skill:")
print(f"Observed Rank Correlation: {observed_corr:.4f}")
print(f"P-value: {perm_p_val:.4f}")
print(f"Prediction Skill Significant: {'YES' if perm_p_val < 0.05 else 'NO'}")

In [ ]:
# Visualize permutation test
fig, ax = plt.subplots(figsize=(10, 5))

ax.hist(null_dist, bins=50, edgecolor='black', alpha=0.7, label='Null Distribution')
ax.axvline(x=observed_corr, color='red', linestyle='--', linewidth=2, 
           label=f'Observed: {observed_corr:.4f}')
ax.axvline(x=np.percentile(null_dist, 95), color='orange', linestyle=':', 
           label='95th Percentile')
ax.set_xlabel('Average Rank Correlation')
ax.set_ylabel('Frequency')
ax.set_title(f'Permutation Test (p={perm_p_val:.4f})')
ax.legend()

plt.tight_layout()
plt.savefig('../outputs/backtest/permutation_test.png', dpi=150)
plt.show()

## 9. Key Findings & Recommendations

In [ ]:
print("="*70)
print("DIAGNOSTIC SUMMARY")
print("="*70)

print("\n1. PREDICTION QUALITY:")
print(f"   - Average rank correlation: {weekly_correlations.mean():.4f}")
print(f"   - % weeks with positive correlation: {(weekly_correlations > 0).mean()*100:.1f}%")
print(f"   - Hit rate (top-3 picks > 0): {pred_quality['hit_rate_top_k']*100:.1f}%")

print("\n2. STATISTICAL SIGNIFICANCE:")
print(f"   - Sharpe 95% CI: [{sig.sharpe_ci_lower:.3f}, {sig.sharpe_ci_upper:.3f}]")
print(f"   - P-value (Sharpe > 0): {sig.sharpe_p_value:.4f}")
print(f"   - Permutation p-value: {perm_p_val:.4f}")

print("\n3. FEATURE ANALYSIS:")
n_significant = feat_corr_df['Significant'].sum()
print(f"   - Features with significant correlation: {n_significant}/{len(feature_cols)}")
if n_significant > 0:
    top_feat = feat_corr_df.iloc[0]
    print(f"   - Strongest feature: {top_feat['Feature']} (r={top_feat['Pearson Corr']:.4f})")

print("\n4. REGIME ANALYSIS:")
for _, row in regime_stats.iterrows():
    print(f"   - {row['regime_value']}: mean={row['mean_return']*100:.2f}%, n={row['n_weeks']} weeks")

print("\n" + "="*70)
print("RECOMMENDATIONS FOR FEATURE ENGINEERING")
print("="*70)
print("""
Based on this analysis, the model struggles because:

1. LOW PREDICTION SKILL
   - Rank correlations are near zero or negative
   - Current features lack predictive power for relative ETF performance
   
2. FEATURE RECOMMENDATIONS
   a) Technical Indicators:
      - Momentum (20d, 60d returns)
      - RSI (Relative Strength Index)
      - Bollinger Band %B (mean reversion signal)
      - ATR (Average True Range) for volatility
   
   b) Cross-Sectional Features:
      - Rank momentum (relative to other ETFs)
      - Sector relative strength
      - Z-score of returns
   
   c) Regime Indicators:
      - VIX level and changes
      - Yield curve slope
      - Credit spreads
   
   d) Fundamental Flows:
      - ETF fund flows (if available)
      - Short interest
      
3. NEXT STEPS
   - Implement technical indicator features
   - Add cross-sectional rank features
   - Test with walk-forward validation
   - If still no signal, consider alternative prediction targets
""")

In [ ]:
print("\nDiagnostic analysis complete!")
print("Figures saved to ../outputs/backtest/")